<a href="https://colab.research.google.com/github/franmazzaro/7506R-2C2023-GRUPO39/blob/main/7506R_TP2_GRUPO39_CHP1_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Librerias**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.feature_selection import SelectPercentile

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Carga de datasets

In [5]:
df_criticas = pd.read_csv('/content/sample_data/train.csv')
df_criticas_test = pd.read_csv('/content/sample_data/test.csv')


In [ ]:
df_criticas.head(4)

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo


# Bayes Naive

Dividimos nuestro conjunto de train en 70 30

In [29]:
x_train, x_test, y_train, y_test = train_test_split(df_criticas.review_es,
                                                    df_criticas.sentimiento,
                                                    test_size=0.30,
                                                    random_state=25,
                                                    shuffle=True)

Antes de entrear un modelo para clasificacion necesitamos que este entrenamiento sea con valores numericos, y como nosotros tenemos palabras necesitamos convertirlas a vectores.

Utilizamos make_pipeline para aplicarle a nuestros modelos dos metodos distintos para vectorizar y luego el modelo de Naive Bayes Multidimensional. Una vez obtenidos los entrenamos

In [30]:
model_NB = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_NB_2 = make_pipeline(CountVectorizer(), MultinomialNB())

model_NB.fit(x_train, y_train)
model_NB_2.fit(x_train, y_train)

categorias_predichas = model_NB.predict(x_test)
categorias_predichas_2 = model_NB_2.predict(x_test)


In [32]:
print(accuracy_score(y_test, categorias_predichas))
print(accuracy_score(y_test, categorias_predichas_2))

0.8534666666666667
0.8376


Ya que el modelo 2 tuvo mejor accuracy es el que utilizamos para realizar la predicción de test

In [ ]:
criticas_test = df_criticas_test['review_es']
categorias_predichas_test = model_NB.predict(criticas_test)


Generamos el archivo de predicciones

In [ ]:
ids_test =  df_criticas_test.ID
filename = 'Naive_Bayes_Multidimensional'
prediccion = pd.DataFrame({"id": ids_test, "sentimiento": categorias_predichas_test})
prediccion.to_csv('/content/sample_data/prediccion_'+filename+'.csv', index = False)

# Random Forest

In [33]:
RF_fid = make_pipeline(TfidfVectorizer(), RandomForestClassifier(n_estimators=100, random_state=42))
RF_count = make_pipeline(CountVectorizer(), RandomForestClassifier(n_estimators=100, random_state=42))

RF_fid.fit(x_train, y_train)
RF_count.fit(x_train, y_train)

categorias_predichas = RF_fid.predict(x_test)
categorias_predichas_2 = RF_count.predict(x_test)

In [36]:
print(classification_report(y_test, categorias_predichas))
print(classification_report(y_test, categorias_predichas_2))

              precision    recall  f1-score   support

    negativo       0.83      0.84      0.84      7599
    positivo       0.83      0.83      0.83      7401

    accuracy                           0.83     15000
   macro avg       0.83      0.83      0.83     15000
weighted avg       0.83      0.83      0.83     15000

              precision    recall  f1-score   support

    negativo       0.84      0.84      0.84      7599
    positivo       0.83      0.84      0.84      7401

    accuracy                           0.84     15000
   macro avg       0.84      0.84      0.84     15000
weighted avg       0.84      0.84      0.84     15000



Ya que el modelo que usa CountVectorizer es ligeramente mejor, usaremos ese para predecir

In [37]:
categorias_predichas_test_2 = RF_count.predict(df_criticas_test['review_es'])

Generamos el archivo de predicciones

In [40]:
filename = 'Random_Forest_CountVectorizer'
prediccion = pd.DataFrame({"id": df_criticas_test.ID, "sentimiento": categorias_predichas_test_2})
prediccion.to_csv('/content/sample_data/prediccion_'+filename+'.csv', index = False)